In [2]:
!pip install gcsfs

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cugraph 24.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
cudf 24.8.3 requires cubinlinker, which is not installed.
cudf 24.8.3 requires cupy-cuda11x>=12.0.0, which is not installed.
cudf 24.8.3 requires ptxcompiler, which is not installed.
dask-cudf 24.8.3 requires cupy-cuda11x>=12.0.0, which is not installed.
cuml 24.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
datasets 3.6.0 requires fsspec[http]<=2025.3.0,>=2023.1.0, but you have fsspec 2025.7.0 which is incompatible.
cudf 24.8.3 requires cuda-python<12.0a0,>=11.7.1, but you have cuda-python 12.6.2.pos

In [1]:
import io
import pandas as pd
import scanpy as sc
import pyarrow.dataset as ds
import gcsfs

In [2]:
# initialize GCS file system for reading data from GCS
fs = gcsfs.GCSFileSystem()

In [3]:
# GCS bucket path
gcp_base_path = "gs://arc-ctc-tahoe100/2025-02-25/"

In [4]:
# set the path to the obs_metadata file
infile = "/".join([gcp_base_path.rstrip("/"), 'metadata', 'obs_metadata.parquet'])
obs_metadata = ds.dataset(infile, filesystem=fs, format="parquet")

In [9]:
scanner = obs_metadata.scanner(columns=["cell_line", 'drugname_drugconc', 'tscp_count'])
obdf_df = scanner.to_table()

In [20]:
median_UMIs = obdf_df.to_pandas().groupby(["cell_line"])['tscp_count'].median()
n_cells = obdf_df.to_pandas().groupby(["cell_line", 'drugname_drugconc']).size()

/tmp/ipykernel_1517846/2924322574.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  median_UMIs = obdf_df.to_pandas().groupby(["cell_line"])['tscp_count'].median()


In [30]:
n_cells_cellline = n_cells.reset_index().groupby('cell_line')[0].median().reset_index()
n_cells_cellline.columns = ['cell_line', 'median_n_cells_x_perturb']
tahoe_cellline_stats = pd.merge(n_cells_cellline, median_UMIs.reset_index())
tahoe_cellline_stats.rename({'tscp_count':'median_tscp_count'}, axis=1).to_csv('../../metadata/Tahoe100M_cellline_stats.csv')

/tmp/ipykernel_1517846/3856042836.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  n_cells_cellline = n_cells.reset_index().groupby('cell_line')[0].median().reset_index()
